In [1]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
dfFac = pd.read_sql("SELECT Game_Id, Date, Description FROM PBP WHERE Event == 'FAC' AND Period == '1' AND Seconds_Elapsed == '0'", conn)
dfFac[['gagneMAJ', 'Reste']] = dfFac.Description.str.split(" won",expand=True,)
dfFac[['Année', 'Reste', 'Reste2']] = dfFac.Date.str.split("-",expand=True)
dfFac = dfFac.drop(["Description", "Reste2"], axis=1)
s = pd.to_numeric(dfFac["Année"])
dfFac["Année"] = s
r = pd.to_numeric(dfFac["Reste"])
dfFac["Reste"] = r
dfFac["debutSaison"] = r.ge(9)
dfFac["SaisonTempo"] = s * dfFac["debutSaison"]
st = dfFac["SaisonTempo"]
dfFac["finSaison"] = st.eq(0)
dfFac["SaisonTempo2"] = (s - 1) * dfFac["finSaison"]
dfFac["Saison"] = dfFac["SaisonTempo"] + dfFac["SaisonTempo2"]
dfFac["Saison"] = dfFac["Saison"].apply(str)
dfFac = dfFac.drop(["Reste", "Année", "debutSaison", "SaisonTempo", "finSaison", "SaisonTempo2"], axis=1)
dfFac["IdMatch"] = dfFac["Saison"] + "0" + dfFac["Game_Id"]

df = pd.read_csv("C:\dev\pool-de-hockey\src\data\scoreMatchs.csv", sep=";")
df[['Année', 'Reste', 'Reste2']] = df.Date.str.split("-",expand=True)
df = df.drop(["Reste2"], axis=1)
s = pd.to_numeric(df["Année"])
df["Année"] = s
r = pd.to_numeric(df["Reste"])
df["Reste"] = r
df["debutSaison"] = r.ge(9)
df["SaisonTempo"] = s * df["debutSaison"]
st = df["SaisonTempo"]
df["finSaison"] = st.eq(0)
df["SaisonTempo2"] = (s - 1) * df["finSaison"]
df["Saison"] = df["SaisonTempo"] + df["SaisonTempo2"]
df["Saison"] = df["Saison"].apply(str)
df["Game_Id"] = df["Game_Id"].apply(str)
df = df.drop(["Reste", "Année", "debutSaison", "SaisonTempo", "finSaison", "SaisonTempo2"], axis=1)
df["IdMatch"] = df["Saison"] + "0" + df["Game_Id"]

dfFinale = dfFac.merge(df, on = "IdMatch")
dfFinale = dfFinale[["Date_x", "gagneMAJ", "gagnant"]]
#dfFinale.to_csv("C:\dev\pool-de-hockey\src\data\corr.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\dev\\pool-de-hockey\\src\\data\\scoreMatchs.csv'

In [4]:
#dfVic = pd.read_sql("SELECT Game_Id, Date, Event, Away_Team, Home_Team, Away_Score, Home_Score FROM PBP WHERE Event == 'GEND'", conn)
#dfVic.to_csv("C:\dev\pool-de-hockey\src\data\scoreMatchs.csv")

In [1]:
from math import *
import math
from pandas import DataFrame


def loiBinom(p, n, x):
    pre = math.factorial(n)/(math.factorial(x)*math.factorial(n - x))
    mid = pow(p, x)
    end = pow((1 - p), (n - x))
    return pre * mid * end

def probGagnerPari(p, n, gagnerAbs=False):
    if gagnerAbs:
        mi = floor(n/2)
        mi = mi + 1
    else:
        mi = ceil(n/2)
    tot = 0
    for i in range (mi, n+1):
        f = loiBinom(p, n, i)
        tot = tot + f
    return tot

def mettreDf(p, n, gagnerAbs=False):
    lis = list(range(1, n+1))
    df = DataFrame (lis, columns=["n"])
    vec = []
    p = 0.5075
    for i in lis:
        probGagner = probGagnerPari(p, i, gagnerAbs)
        vec.append(probGagner)
    df["probGagner"] = vec
    return df

In [16]:
lis = []
df = mettreDf(0.5075, 1001, True)
df = df.sort_values(by="probGagner", ascending=False)
df

,n,probGagner
1,2,0.757444
3,4,0.698665
999,1000,0.693609
997,998,0.693454
995,996,0.693298
...,...,...
8,9,0.518451
6,7,0.516403
4,5,0.514060
2,3,0.511249


In [2]:
loiBinom(0.2, 100, 24)

0.057733990380810424

In [19]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
master_buts = pd.read_sql("SELECT Game_Id, Date, Period, Seconds_Elapsed, Event, Ev_team, Away_Team, Home_Team, Away_Goalie, Home_Goalie FROM PBP WHERE Event == 'GOAL'", conn)

#ajout colonne comptéSur
master_buts["comptéSur"] = np.where(master_buts["Ev_Team"]==master_buts["Away_Team"], master_buts["Home_Goalie"], master_buts["Away_Goalie"])

#obtenir la df des shifts des gardiens
#uniq = pd.unique(master_buts[['Away_Goalie', 'Home_Goalie']].values.ravel('K'))
#uniqueGar = uniq.tolist()
#n = len(uniqueGar)
#shifts = pd.DataFrame(columns=["Game_Id", "Period", "Team", "Player", "Player_Id", "Start", "End", "Duration", "Date"])
#for i in range(0, n):
 #   gardien = uniqueGar[i]
  #  try:
   #     sql = "SELECT * FROM SHIFTS WHERE Player == '" + gardien + "'"
    #    tempo = pd.read_sql(sql, conn)
     #   shifts = shifts.append(tempo)
    #except:
    #    print("vide")

In [27]:
shifts["Game_Id"] = shifts["Game_Id"].map(str)
shifts["Period"] = shifts["Period"].map(str)
shifts["Date"] = shifts["Date"].map(str)
idGame = shifts["Game_Id"]
periode = shifts["Period"]
date = shifts["Date"]
shifts["matchPeriode"] = periode + idGame + date
shifts["idShift"] = shifts.index
shifts.head()

,Game_Id,Period,Team,Player,Player_Id,Start,End,Duration,Date,Periode,matchPeriode,idShift
0,20001,1,OTT,CRAIG ANDERSON,8467950,0.0,1200.0,1200.0,2019-10-02,1,1200012019-10-02,0
1,20001,2,OTT,CRAIG ANDERSON,8467950,0.0,1200.0,1200.0,2019-10-02,2,2200012019-10-02,1
2,20001,3,OTT,CRAIG ANDERSON,8467950,0.0,1083.0,1083.0,2019-10-02,3,3200012019-10-02,2
3,20001,3,OTT,CRAIG ANDERSON,8467950,1134.0,1158.0,24.0,2019-10-02,3,3200012019-10-02,3
4,20020,1,OTT,CRAIG ANDERSON,8467950,0.0,1200.0,1200.0,2019-10-05,1,1200202019-10-05,4


In [26]:
master_buts["Game_Id"] = master_buts["Game_Id"].map(str)
master_buts["Period"] = master_buts["Period"].map(str)
master_buts["Date"] = master_buts["Date"].map(str)
idGame = master_buts["Game_Id"]
periode = master_buts["Period"]
date = master_buts["Date"]
master_buts["matchPeriode"] = periode + idGame + date
master_buts.head()

,Game_Id,Date,Period,Seconds_Elapsed,Event,Ev_Team,Away_Team,Home_Team,Away_Goalie,Home_Goalie,comptéSur,matchPeriode
0,20001,2019-10-02,1,25.0,GOAL,OTT,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,FREDERIK ANDERSEN,1200012019-10-02
1,20001,2019-10-02,2,140.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02
2,20001,2019-10-02,2,282.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02
3,20001,2019-10-02,2,351.0,GOAL,OTT,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,FREDERIK ANDERSEN,2200012019-10-02
4,20001,2019-10-02,2,482.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02


In [30]:
gardien = "CRAIG ANDERSON"
per = "2200012019-10-02"
is_period = master_buts["matchPeriode"] == per
df = master_buts[is_period]
is_gar = df["comptéSur"] == gardien
df[is_gar]

,Game_Id,Date,Period,Seconds_Elapsed,Event,Ev_Team,Away_Team,Home_Team,Away_Goalie,Home_Goalie,comptéSur,matchPeriode
1,20001,2019-10-02,2,140.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02
2,20001,2019-10-02,2,282.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02
4,20001,2019-10-02,2,482.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02
5,20001,2019-10-02,2,890.0,GOAL,TOR,OTT,TOR,CRAIG ANDERSON,FREDERIK ANDERSEN,CRAIG ANDERSON,2200012019-10-02
